In [ ]:
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/Happymonk/task3/data.rar

Archive:  /content/drive/MyDrive/Happymonk/task3/data.rar
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/Happymonk/task3/data.rar or
        /content/drive/MyDrive/Happymonk/task3/data.rar.zip, and cannot find /content/drive/MyDrive/Happymonk/task3/data.rar.ZIP, period.


In [ ]:
!pip install unrar
!unrar x /content/drive/MyDrive/Happymonk/task3/data.rar

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image
import os

In [ ]:
list_subfolders_with_paths

['/content/data/train/dog',
 '/content/data/train/cat',
 '/content/data/train/.ipynb_checkpoints']

In [ ]:
alphabets = {}

# Get image from dataset
base_dir = '/content/data/'
train_dir = os.path.join(base_dir, 'train')

list_subfolders_with_paths = [f.path for f in os.scandir(train_dir) if f.is_dir()]

for i in list_subfolders_with_paths:
    # print(i)
    # Ignore moving alphabet
    # if i.split('///')[0] in ['cat','dog']:
    alphabets[i.split('///')[0]] = [f.path for f in os.scandir(i) if f.path.endswith('.jpg')]

dataset = pd.DataFrame.from_dict(alphabets)

ValueError: ignored

In [ ]:
dataset

""


In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('/content/data/train/cat/cat.1.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/content/data/train/cat_aug', save_prefix='cat', save_format='jpg'):
    i += 1
    if i > 10:
        break  